In [54]:
import requests
import json
import numpy as np
import csv
API_KEY = "AVMHeV6pxPlMksujAClTZYB9ylNpKnKdJWLUOVDogtwz"


In [21]:
def get_stories(x):
    BASE_URL = "https://www.wattpad.com/v4/stories?limit=100offset%3D0&offset="+str(x)

    header = {
    "Authorization": "Basic {}".format(API_KEY),
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36",

    }

    req = requests.get(BASE_URL.format("stories"), headers=header)
    json_response = req.json()
    return(json_response)
    #print(json.dumps(json_response, indent=3, sort_keys=True)) 

In [40]:
#number of stories
N=10000
json_list=[]
for x in np.arange(0,N,100):
    json_list.append(get_stories(x))

In [50]:
pages_of_stories=[x['stories'] for x in json_list]
print(len(pages_of_stories))
pages_of_stories[10]

100


[{'categories': [19, 0],
  'commentCount': 0,
  'completed': False,
  'copyright': 1,
  'cover': 'https://a.wattpad.com/cover/115611899-256-k686082.jpg',
  'cover_timestamp': '2017-07-10T18:28:02Z',
  'createDate': '2017-07-10T17:42:55Z',
  'deleted': False,
  'description': 'Yaşam ve ölümün arasındaki ipi anında kesen tek makas, tetiktir.\n\nDolambaçlı hayatına bir de "ölüm" kavramını ekleyen Arif Tektan, Kara Harp Okulu\'ndan mezun olduktan sonra Kuzey Irak\'ta aldığı görevlerin etkisiyle Üsteğmen olarak Özel Kuvvetler Komutanlığı\'na başvurur. Güneydoğu\'nun en ıssız köşelerine dahi bakmakla görevlendirilen Arif, dağlarda geçireceği heyecan dolu her dakika için ömründen bir yıl kaybedecek. O ülkesi için savaşmasaydı, ülkesi olmayacaktı. Arif, canı pahasına ülkesini savunmayı seçti.',
  'firstPartId': 438651983,
  'firstPublishedPart': {'createDate': '2017-07-10T18:28:03Z',
   'id': 438651983},
  'id': '115611899',
  'language': {'id': 1, 'name': 'English'},
  'lastPublishedPart': {'

In [53]:
flat_list=[x for y in pages_of_stories for x in y]
len(flat_list)
flat_list[40]

{'categories': [19, 0],
 'commentCount': 28,
 'completed': True,
 'copyright': 1,
 'cover': 'https://a.wattpad.com/cover/124718911-256-k480463.jpg',
 'cover_timestamp': '2017-10-08T23:34:23Z',
 'createDate': '2017-10-05T06:01:50Z',
 'deleted': False,
 'description': 'Severus and Hermione love story. A silly little game, and a whole lot of trouble. A playful pet name and some loving sarcasm. Perfect combination for some love.',
 'firstPartId': 478089673,
 'firstPublishedPart': {'createDate': '2017-10-11T01:40:30Z', 'id': 478089673},
 'id': '124718911',
 'language': {'id': 1, 'name': 'English'},
 'lastPublishedPart': {'createDate': '2017-10-11T08:41:15Z', 'id': 478097551},
 'length': 83873,
 'mature': False,
 'modifyDate': '2017-10-11T01:41:59Z',
 'numParts': 10,
 'parts': [{'commentCount': 15,
   'draft': False,
   'id': 478089673,
   'length': 6413,
   'modifyDate': '2017-10-11T01:40:30Z',
   'photoUrl': 'https://d.wattpad.com/story_parts/478089673/images/14ea97a2f95ab303650525023182.j

In [56]:
with open('flat_list.csv','w') as file:
    write=csv.writer(file)
    write.writerow(flat_list)